# Exploration_4
## Romantic AI Lyricist

1. 데이터 다운로드
2. 데이터 읽어오기 : glob모듈 사용하여 모든 txt파일을 읽어온 후, raw_corpus리스트에 문장 단위로 저장.
3. 데이터 정제 : preprocess_sentence()함수를 활용해 데이터 정제. 지나치게 긴 문장은 다른 데이터들이 과도한 padding을 갖게 하므로 제거.(문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외)
4. 평가 데이터셋 분리 : tokenize()함수로 데이터를 Tensor로 변환한 후, sklearn모듈의 train_test_split()함수를 사용해 훈련 데이터와 평가 데이터를 분리.(단어장의 크기는 12,000이상으로 설정, 총 데이터의 20%를 평가 데이터셋으로)
5. 인공지능 만들기 : 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch안에 val_loss값을 2.2수준으로 줄일 수 있는 모델 설계

## 순환신경망(RNN)
* (start)라는 특수한 토큰을 맨 앞에 추가해줌. 인공지능에게 문장을 생성하라는 사인을 주는 셈.
* (end)라는 특수한 토큰을 마지막에 추가해줌. 인공지능이 다 만들었다는 사인으로 생성.
* (start)를 입력으로 받은 순환신경망은 다음 단어를 생성하고, 생성한 단어를 다시 입력으로 사용함. 이런 순환적인 특성을 살려 순환신경망이라고 함.
* 즉, (start)가 문장의 시작에 더해진 입력 데이터(문제지)와, (end)가 문장의 끝에 더해진 출력 데이터(답안지)가 필요. 문장 데이터만 있으면 만들어낼 수 있다.

### (1) 데이터 다운로드

### (2) 데이터 읽어오기
* glob모듈 사용하여 모든 txt파일을 읽어온 후, raw_corpus리스트에 문장 단위로 저장.

In [1]:
import glob
import os
import re # regex(정규표현식)
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    # with문은 close를 쓰지 않아도 자동으로 반환해준다.
    with open(txt_file, "r") as f: # r:읽기모드(파일을 읽기만 할 때)
        raw = f.read().splitlines() # 라인 단위로 끊어서 list 형태로 읽어온다.
        raw_corpus.extend(raw)

# 데이터 크기도 알아보자!
print("데이터 크기:", len(raw_corpus))
# 앞에서부터 10라인만 출력해보자!
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


### (3) 데이터 정제
* preprocess_sentence()함수를 활용해 데이터 정제. 지나치게 긴 문장은 다른 데이터들이 과도한 padding을 갖게 하므로 제거.(문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외)

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿']+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

#### 정규표현식(regex)을 통한 문장정리
1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿'가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 (start), 끝에는 (end)를 추가합니다
* 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
---
* 전처리를 위해 정규표현식(Regex)을 이용한 필터링이 유용하게 사용.
* whitespace : 띄어쓰기(' '), 탭('₩t'), 엔터('₩n')
* 이 whitespace를 제거하기 위해서 strip함수를 사용.
* 문자열의 끝에 .strip()을 붙이면, 문자열의 '맨앞'과 '맨뒤'의 whitespace가 제거됨. 그러나 중간의 whitespace는 제거되지 않음.

In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

["<start> now i've heard there was a secret chord <end>",
 '<start> that david played , and it pleased the lord <end>',
 "<start> but you don't really care for music , do you ? <end>",
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [4]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    # 아래의 tokenizer패키지는 정제된 데이터를 토큰화하고, 단어사전을 만들어주며, 숫자로 변환된 데이터를 텐서라고 칭함.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   46  149 ...    0    0    0]
 [   2   15 2768 ...    0    0    0]
 [   2   32    7 ...    3    0    0]
 ...
 [   2  129   20 ...   10 1040    3]
 [8545    6   35 ... 1312  656    3]
 [   2    7   30 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f032953aac0>


* 토큰화(Tokenize) : 문장을 일정한 기준으로 쪼개는 과정
* 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외

In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : a
10 : to


In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다.
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
src_input = tensor[:, :-1] 
# tensor에서 <start>를 잘라내서 타겟 문장을 생성한다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   46  149  302   86   53    9 1006 6212    3    0    0    0    0]
[  46  149  302   86   53    9 1006 6212    3    0    0    0    0    0]


### (4) 평가 데이터셋 분리
* tokenize()함수로 데이터를 Tensor로 변환한 후, sklearn모듈의 train_test_split()함수를 사용해 훈련 데이터와 평가 데이터를 분리.(단어장의 크기는 12,000이상으로 설정, 총 데이터의 20%를 평가 데이터셋으로)

In [7]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size=0.2,
                                                         random_state=7)
print(len(src_input))
print(len(tgt_input))
print('입력 훈련데이터 : ', enc_train.shape)
print('입력 검증데이터 : ', enc_val.shape)
print('출력 훈련데이터 : ', dec_train.shape)
print('출력 검증데이터 : ', dec_val.shape)

175986
175986
입력 훈련데이터 :  (140788, 14)
입력 검증데이터 :  (35198, 14)
출력 훈련데이터 :  (140788, 14)
출력 검증데이터 :  (35198, 14)


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) # drop_remainder : 마지막 남은 데이터를 drop할 것인지 여부
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### (5) 인공지능 만들기
* 5. 인공지능 만들기 : 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch안에 val_loss값을 2.2수준으로 줄일 수 있는 모델 설계

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(enc_train, dec_train,validation_data=(enc_val, dec_val), epochs=10, batch_size=256)

Epoch 1/10
550/550 [==============================] - 111s 191ms/step - loss: 3.6326 - val_loss: 3.2892
Epoch 2/10
550/550 [==============================] - 109s 199ms/step - loss: 3.1659 - val_loss: 3.0814
Epoch 3/10
550/550 [==============================] - 110s 200ms/step - loss: 2.9712 - val_loss: 2.9517
Epoch 4/10
550/550 [==============================] - 110s 200ms/step - loss: 2.8196 - val_loss: 2.8556
Epoch 5/10
550/550 [==============================] - 110s 200ms/step - loss: 2.6857 - val_loss: 2.7792
Epoch 6/10
550/550 [==============================] - 110s 201ms/step - loss: 2.5623 - val_loss: 2.7193
Epoch 7/10
550/550 [==============================] - 110s 200ms/step - loss: 2.4484 - val_loss: 2.6707
Epoch 8/10
550/550 [==============================] - 110s 200ms/step - loss: 2.3413 - val_loss: 2.6284
Epoch 9/10
550/550 [==============================] - 110s 200ms/step - loss: 2.2390 - val_loss: 2.5949
Epoch 10/10
550/550 [==============================] - 110s 200m

* Embedding size : 512
* Hidden size : 1024 (LSTM layer의 hidden state의 차원수. output의 크기)
* epoch : 10
* val_loss : 2.1420

In [11]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  6295552   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


#### Embedding?
* 자연어는 수치화되어 있지 않은 데이터이기 때문에 머신러닝이나 딥러닝 기법을 바로 사용할 수 없다. 그래서 자연어 처리에서 특징 추출을 통해 수치화를 해줘야 하는데 이 때 사용하는 것이 "언어의 벡터화"이다. 이 벡터화의 과정을 Word Embedding이라고 한다.
* Embedding layer : 입력 텐서에는 단어 사전의 인덱스가 들어있는데, 이 인덱스 값을 해당 인덱스 번째의 워드벡터로 바꿔주는 역할을 함. 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용.
* embedding size : 워드 벡터의 차원수. 즉 단어가 추상적으로 표현되는 크기. 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만 야기할 수 있음.

In [16]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [26]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you , i love you <end> '

### (6) 회고
#### 아쉬운점
1. 모델이 생성한 가사가 사랑꾼이긴한데... 너무 단순해서 좀 아쉬웠다. 그래서 init_sentence에 다양한 문장도입부를 입력하여 여러 출력값을 도출해보았다. 'i know'로 바꿔서 'i know you want to be'라는 결과를 얻어보았고, 'i need'는 'i need you to hurry up now', 'i want'는 'i want to be your lover'와 같은 가사들을 얻을 수 있었다. 
---
#### 배운점
1. compile과 fit 과정이 어떤 과정 다음 순서로 오는지 제대로 이해하지 못해서, 모델을 학습시키기 전에 많은 시간이 걸렸다.
2. embedding size의 경우 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기할 수 있기 때문에 여러번의 시도 끝에 (1024->256->)512란 값을 지정해주었다.
3. hidden size는 모델에 필요한 일꾼 수 같은 개념인데, 각각의 일꾼들이 모두 같은 데이터를 보고 각자의 생각을 가지기 떄문에 embedding size와 마찬가지로 충분한 데이터가 주어지면 올바른 결정을 내리지만 그렇지 않으면 엉뚱한 길로 세버린다. 따라서 hidden size 또한 여러번의 시도 끝에 (4096->2048->)1024란 값을 지정해주었다.